# Agent 구축 가이드
Langchain 주요 사용 기법인 에이전트 구축 방법을 기술한다.

## 라이브러리 설치
Langchain 사용을 위한 필수 라이브러리를 설치한다.

In [ ]:
!pip install langchain langchain-openai langchain-community langgraph langchain-anthropic tavily-python

## LangSmith 연동
LangSmith를 통해 흐름을 분석하길 원한다면 아래 변수를 설정하여 연동한다.

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "Insert Your LangSmith API Key"

## Tavily
검색 엔진 도구로 [Tavily](https://app.tavily.com/home)를 사용하기 위해 API키를 설정한다.

In [ ]:
os.environ["TAVILY_API_KEY"]="Insert Your Tavily API Key"

### Tool을 정의합니다.
에이전트가 사용할 툴을 생성해야한다. LangChain에 내장되어있는 tavily_search 툴을 사용한다.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Seoul")
print(search_results)
tools = [search]

## LLM 정의
LangChain에서 다양한 언어 모델을 지원하지만 여기서는 OpenAI 모델을 사용한다.

In [ ]:
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "Insert Your OpenAI API Key"
model = ChatOpenAI(model="gpt-4o")

언어 모델을 호출할 때 메시지 목록을 전달한다.

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

`.bind_tools`를 사용하여 언어 모델이 툴에 대한 지식을 갖도록 한다.

In [ ]:
model_with_tools = model.bind_tools(tools)

모델을 호출하기 위해 일반 메시지로 호출한다.

`content` 필드와 `tool_calls` 필드 모두 확인 가능하다.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


툴 호출이 필요한 입력으로 호출한다.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Seoul?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

## 에이전트 생성
툴과 LLM을 정의했으므로 에이전트를 생성할 수 있다.

여기서는 [LangGraph](https://langchain-ai.github.io/langgraph/)를 사용하여 에이전트를 구성한다.
`LangGraph`를 사용하면 간편하게 로직을 수정할 수 있다.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## 에이전트 실행
에이전트를 실행합니다. 에이전트는 상호작용의 최종 상태를 반환한다.

In [ ]:
# 툴 호출 필요없을 때 응답
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

In [ ]:
# 툴 호출 필요할 때 응답
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Seoul?")]}
)
response["messages"]

## 메시지 스트리밍
`.invoke`를 사용하여 최종 응답만을 가져오면 에이전트가 여러 단계를 실행할 때 오랜 시간을 기다리게 된다.

중간 진행 상황을 표시하여 메시지를 스트리밍 할 수 있다.

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Seoul?")]}
):
    print(chunk)
    print("----")

## 토큰 스트리밍
메시지를 스트리밍하는 것 외에도 토큰을 스트리밍하는 것도 유용하다.

`.astream_events` 메서드를 사용하여 이를 수행할 수 있다.

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Seoul?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # 에이전트를 `.with_config({"run_name": "Agent"})`로 생성할 때 지정됨
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # 에이전트를 `.with_config({"run_name": "Agent"})`로 생성할 때 지정됨
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # OpenAI의 문맥에서 비어 있는 내용은
            # 모델이 호출할 도구를 요청하고 있음을 의미합니다.
            # 따라서 빈 내용이 아닌 내용만 인쇄합니다.
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

## 메모리 추가
지금까지 구현 한 내용으로는 이전 대화를 기억하지 않기 때문에 자연스러운 대화가 이어질 수 없다.

메모리를 제공하기 위해 체크포인터를 전달한다.

`thread_id`를 전달하여 이어지는 대화인지, 새로운 대화인지 구분한다.

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

새로운 대화를 시작할 때는 `thread_id`를 변경한다.

In [ ]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")